# Deploy Specific Model Version to Red Hat OpenShift Data Science

This notebook deploys the model.joblib file to OpenShift Data Science through Inference Service.
It is required that the model.joblib and inference-service.yaml file exists.
In the pipeline, the model.joblib file is generated by the preceding job.

import os, time, subprocess
from jinja2 import Template

In [32]:
import os, time, subprocess
from jinja2 import Template

variables = {}
with open("vars.txt") as myfile:
    for line in myfile:
        name, var = line.partition("=")[::2]
        variables[name.strip()] = str(var)

In [ ]:
storage_key="aws-connection-fraud-detection"
template_data = {"model_version": variables['model_version'], "storage_key": storage_key}

def deploy_template(filename, template_data):
    print("invoking template:" + filename)
    template = Template(open(filename).read())
    rendered_template = template.render(template_data)
    
    subprocess.run(['oc', 'whoami'])
    ps = subprocess.Popen(['echo', rendered_template], stdout=subprocess.PIPE)
    print(ps.stdout)
    output = subprocess.check_output(['oc', 'apply', '-f', '-'], stdin=ps.stdout)
    ps.wait()
    print(f"Deployed template {filename}. Version: {variables['model_version']}")
    
templates = ['deployment/serving-runtime.yaml','deployment/inference-service.yaml']
for t in templates:
    deploy_template(t, template_data)